In [1]:
import pandas as pd

In [2]:
high_voltage_vertices_df = pd.read_csv('data/gridkit_europe-highvoltage-vertices.csv')

In [3]:
high_voltage_vertices_df.head()

,v_id,lon,lat,typ,voltage,frequency,name,operator,ref,wkt_srid_4326
0,27978,-2.176592,57.281849,joint,275000,NaN,NaN,NaN,NaN,SRID=4326;POINT(-2.17659206194142 57.281849486...
1,45518,38.215405,48.066401,substation,330000;110000;10000,50,Харцизька-330,NaN,NaN,SRID=4326;POINT(38.2154050764474 48.0664012417...
2,33655,7.704050,51.684767,joint,380000;0,50,380kV Gersteinwerk Ost+Uentrop Nord,RWE,4325,SRID=4326;POINT(7.70404979297499 51.6847670848...
3,44535,-0.858256,41.571922,substation,220000,50,Subestacion Electrica Aceralia,NaN,NaN,SRID=4326;POINT(-0.858256184668034 41.57192238...
4,41625,3.045074,50.543257,substation,NaN,NaN,NaN,NaN,NaN,SRID=4326;POINT(3.04507430374696 50.5432568554...


In [4]:
high_voltage_links_df = pd.read_csv('data/gridkit_europe-highvoltage-links.csv')

In [5]:
high_voltage_links_df.head()

,l_id,v_id_1,v_id_2,voltage,cables,wires,frequency,name,operator,ref,length_m,r_ohmkm,x_ohmkm,c_nfkm,i_th_max_a,from_relation,wkt_srid_4326
0,22139,43193,23620,220000;132000,9,2,50,NaN,ATEL; BKW,NaN,489.727561,NaN,NaN,NaN,NaN,NaN,SRID=4326;LINESTRING(7.61602141237887 47.23259...
1,65908,42022,13686,750000,3;1;1;1;3,4;4;4;4,NaN,750 kV,NaN,NaN,394971.083012,NaN,NaN,NaN,NaN,NaN,SRID=4326;LINESTRING(26.6449735029238 50.29628...
2,67370,6913,48526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10602.245828,NaN,NaN,NaN,NaN,NaN,SRID=4326;LINESTRING(18.1844471738857 40.01608...
3,3868,35422,28973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2385.934530,NaN,NaN,NaN,NaN,NaN,SRID=4326;LINESTRING(10.2724738664462 49.88060...
4,4982,7864,63104,225000,3,1,50,NaN,RTE,NaN,1449.986592,NaN,NaN,NaN,NaN,NaN,SRID=4326;LINESTRING(3.80033661099728 45.04510...


In [6]:
import folium
import pandas as pd
import networkx as nx

# Initialize an undirected graph
G = nx.Graph()

# Create a base map centered on a specific region in Europe (e.g., Paris, France)
center_lat = 48.8566  # Latitude for Paris
center_lon = 2.3522  # Longitude for Paris
map = folium.Map(location=[center_lat, center_lon], zoom_start=10)  # Zoomed in to a smaller region

# Define a function to limit the vertices to a specific latitude/longitude range
def filter_by_region(df, lat_min, lat_max, lon_min, lon_max):
    return df[(df['lat'] >= lat_min) & (df['lat'] <= lat_max) & (df['lon'] >= lon_min) & (df['lon'] <= lon_max)]

# Define the bounding box for the region (latitude and longitude ranges)
lat_min, lat_max = 35.0, 60.0
lon_min, lon_max = -10.0, 30.0

# Filter the vertices to only include those within the bounding box
filtered_vertices_df = filter_by_region(high_voltage_vertices_df, lat_min, lat_max, lon_min, lon_max)

# Filter the links to only include those within the bounding box
filtered_links_df = high_voltage_links_df[
    (high_voltage_links_df['v_id_1'].isin(filtered_vertices_df['v_id'])) & 
    (high_voltage_links_df['v_id_2'].isin(filtered_vertices_df['v_id']))
]

# Add vertices to the graph
for index, row in filtered_vertices_df.iterrows():
    # Add the vertex to the graph (v_id as the node ID)
    G.add_node(row['v_id'], pos=(row['lat'], row['lon']))

# Add edges to the graph
for index, row in filtered_links_df.iterrows():
    # Get the vertex IDs for the edge
    v_id_1 = row['v_id_1']
    v_id_2 = row['v_id_2']
    
    # Add the edge to the graph (undirected)
    G.add_edge(v_id_1, v_id_2)

# Now, remove nodes with a degree less than 5
nodes_to_remove = [node for node, degree in dict(G.degree()).items() if degree < 5]
G.remove_nodes_from(nodes_to_remove)

# Visualize the remaining nodes and edges on the folium map

# Add the remaining vertices to the map
for node in G.nodes:
    node_data = filtered_vertices_df[filtered_vertices_df['v_id'] == node].iloc[0]
    folium.Marker(
        location=[node_data['lat'], node_data['lon']],
        popup=f"Vertex ID: {node}",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(map)

# Add the remaining edges to the map
for u, v in G.edges:
    vertex_1 = filtered_vertices_df[filtered_vertices_df['v_id'] == u].iloc[0]
    vertex_2 = filtered_vertices_df[filtered_vertices_df['v_id'] == v].iloc[0]
    
    folium.PolyLine(
        locations=[(vertex_1['lat'], vertex_1['lon']), (vertex_2['lat'], vertex_2['lon'])],
        color="green",  # Color of the edges
        weight=2.5,  # Thickness of the line
        opacity=1.0,
        popup=f"Edge connecting Vertex ID: {u} and Vertex ID: {v}"
    ).add_to(map)

# Save the map to an HTML file
map.save("filtered_power_grid_network_with_edges.html")

# If you're using a Jupyter Notebook, display the map inline:
map
